In [ ]:
!pip install fasttext

     |████████████████████████████████| 71kB 2.1MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3020504 sha256=610ef8b330ee5ac956e2495a1ea0b5c42eb52cab90f21529f6a803bdc4370b23
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 6.2MB/s 


In [ ]:
#if you using google drive:
#!pip install fasttext
#!pip install pymorphy2

#import gzip
#import shutil
#from google.colab import drive

#drive.mount('/content/drive')
#source=r"drive/My Drive/fast_text_models/cc.ru.300.bin.gz"
#output=r"cc.ru.300.bin"
#with gzip.open(source,"rb") as f_in, open(output,"wb") as f_out:
#    shutil.copyfileobj(f_in, f_out)

#ft_ru = fasttext.load_model('cc.ru.300.bin')   

In [ ]:
import fasttext
import pymorphy2
import re
from scipy import spatial
import numpy as np
from tqdm import tqdm

import gzip
import shutil

In [ ]:
source=r"Data/fast_text_models/cc.ru.300.bin.gz"
output=r"cc.ru.300.bin"
with gzip.open(source,"rb") as f_in, open(output,"wb") as f_out:
    shutil.copyfileobj(f_in, f_out)

In [ ]:
ft_ru = fasttext.load_model('cc.ru.300.bin')

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
dict_queries = dict()
with open('Data/queries_b.txt','r', encoding='utf-8') as f:
    for line in f.readlines():
        tmp = line.strip().split('\t')
        if tmp[0] not in dict_queries:
            tmp_w = tmp[1].split(' ')
            norm_form =[]
            for w in tmp_w:
                norm_form.append(morph.parse(w)[0].normal_form)
            norm_form = ' '.join(norm_form)
            dict_queries[tmp[0]] = norm_form

In [ ]:
dict_titles = dict()
with open('Data/titles_norm.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        tmp = line.strip().split('\t')
        dict_titles[tmp[0]] = tmp[1].lower()

In [ ]:
regex = re.compile('[qwertyuiopasdfghjklzxcvbnm]') 

In [ ]:
def get_wordembedding(word):
    word_embedding = ft_ru[word]
    return word_embedding

In [ ]:
def get_words_sim_dict(string_q, string_t):
    res_dict = dict()
    for word_q in string_q.split(' '):
        a = get_wordembedding(word_q)
        for word_t in string_t.split(' '):
            b = get_wordembedding(word_t)
            res_dict[word_q + '\t'+ word_t] = 1-spatial.distance.cosine(a, b)
    return res_dict

In [ ]:
def get_result(query, title, res_dict):
    sorted_dict = sorted(res_dict.items(), key=lambda kv: -1*kv[1])
    result = 0.0
    nums = 0.0
    for x in sorted_dict:
        words = x[0].split('\t')
        score = x[1]
        if words[0] in query and words[1] in title:
            result += score
            nums += 1
            query = query.replace(words[0], '')
            title = title.replace(words[1], '')
            #print(query, title)
        if nums == 0.0:
            return 0.0
    return result / nums

In [ ]:
X_split = []
query_split = []
with open('Data/train.marks.tsv', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        tmp = line.strip().split('\t')
        query_split.append(tmp[0])
        X_split.append(tmp[1])

with open('Data/sample.csv', 'r', encoding='utf-8') as f:
    f.readline()
    for line in f.readlines():
        tmp = line.strip().split(',')
        query_split.append(tmp[0])
        X_split.append(tmp[1])

In [ ]:
X_split = np.array(X_split)

In [ ]:
with open('feat_title/fasttextfeat.txt','w',encoding='utf-8') as f:
    for i in tqdm(range(len(X_split))):
        if (X_split[i] in dict_titles) and (query_split[i] in dict_queries):
            query = dict_queries[query_split[i]]
            title = dict_titles[X_split[i]]
            res_dict = get_words_sim_dict(query, title)
            res_ = get_result(query, title, res_dict)
            f.write(query_split[i] + '\t' + X_split[i] + '\t'+str(res_)+'\n')
        else:
            f.write(query_split[i] + '\t' + X_split[i] + '\t0.0'+'\n') #???

  1%|▏         | 7870/606050 [00:38<40:13, 247.84it/s]/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 606050/606050 [44:08<00:00, 228.86it/s]
